In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# Montar o Drive (faça isso só uma vez por sessão)
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o arquivo CSV no Drive
caminho_csv = '/content/drive/MyDrive/Dataset/fechamento_acoes_ultimo_ano.csv'

# Carregar o CSV
df = pd.read_csv(caminho_csv, index_col=0)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Célula 2 - Carregar e preparar os dados
data = df.values  # (num_dias, 5)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def criar_sequencias(data, passos=30):
    X, y = [], []
    for i in range(len(data) - passos):
        X.append(data[i:i+passos])
        y.append(data[i+passos])
    return np.array(X), np.array(y)

passos = 30
X, y = criar_sequencias(data_scaled, passos)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [9]:
# Célula 3 - Definição do modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=50, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.linear(out)
        return out

model = LSTMModel()


In [10]:
# Célula 4 - Configuração da perda e otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [11]:
# Célula 5 - Função para treinar o modelo
def treinar_modelo(model, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * X_batch.size(0)
        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.6f}")


In [12]:
# Célula 6 - Função para avaliar o modelo
def avaliar_modelo(model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            loss = criterion(output, y_batch)
            total_loss += loss.item() * X_batch.size(0)
    avg_loss = total_loss / len(test_loader.dataset)
    print(f"Loss no teste: {avg_loss:.6f}")
    return avg_loss


In [13]:
# Célula 7 - Treinar e avaliar
treinar_modelo(model, optimizer, epochs=10)
avaliar_modelo(model)


Epoch 1/10 - Loss: 0.387471
Epoch 2/10 - Loss: 0.317514
Epoch 3/10 - Loss: 0.236915
Epoch 4/10 - Loss: 0.128592
Epoch 5/10 - Loss: 0.042578
Epoch 6/10 - Loss: 0.045368
Epoch 7/10 - Loss: 0.030371
Epoch 8/10 - Loss: 0.032717
Epoch 9/10 - Loss: 0.028847
Epoch 10/10 - Loss: 0.028569
Loss no teste: 0.031427


0.03142687678337097